In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_17_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-11 11:00:03.202038


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_17_02/m_17_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_17_02/17_2a_posner_task_2023-06-19_11h49.11.994/17_2a_posner_task_2023-06-19_11h49.11.994.csv
Found '2b' in file: m_17_02/17_2b_posner_task_2023-06-19_12h43.17.233/17_2b_posner_task_2023-06-19_12h43.17.233.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
len(events) 419
len(validity) 100
# of stim events 100
[    0     0 99999]
[27927     0     5]
[27928     0     1]
[27936     0    10]
[31238     0    40]
[32698     0    70]
[36198 

/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [271] Keeping  (2023-06-19 11:59:18.595011+00:00 - 2023-06-19 11:59:18.595011+00:00 -> 469.871 - 469.871)
  [272] Keeping  (2023-06-19 11:59:21.895011+00:00 - 2023-06-19 11:59:21.895011+00:00 -> 473.171 - 473.171)
  [273] Keeping  (2023-06-19 11:59:21.995011+00:00 - 2023-06-19 11:59:21.995011+00:00 -> 473.271 - 473.271)
  [274] Keeping  (2023-06-19 11:59:23.605011+00:00 - 2023-06-19 11:59:23.605011+00:00 -> 474.881 - 474.881)
  [275] Keeping  (2023-06-19 11:59:25.105011+00:00 - 2023-06-19 11:59:25.105011+00:00 -> 476.381 - 476.381)
  [276] Keeping  (2023-06-19 11:59:28.195011+00:00 - 2023-06-19 11:59:28.195011+00:00 -> 479.471 - 479.471)
  [277] Keeping  (2023-06-19 11:59:28.295011+00:00 - 2023-06-19 11:59:28.295011+00:00 -> 479.571 - 479.571)
  [278] Keeping  (2023-06-19 11:59:30.105011+00:00 - 2023-06-19 11:59:30.105011+00:00 -> 481.381 - 481.381)
  [279] Keeping  (2023-06-19 11:59:31.605011+00:00 - 2023-06-19 11:59:31.605011+00:00 -> 482.881 - 482.881)
  [280] Keeping  (2023-06-19

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['1', '10', '101', '109', '117', '20', '30', '40', '5', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
419 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 419 events and 701 original time points ...
8 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.8s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 47.9s.


Method,fastica
Fit,78 iterations on raw data (675772 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=675772
    Range : 31238 ... 707009 =     31.238 ...   707.009 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
337 matching events found
No baseline correction applied
0 projection items ac

In [7]:
print(ica.exclude)

[0]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 1 ICA component
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_17_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/cleaned/m_17_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_17_02/m_17_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_17_02
├── m_17_02_pos2a.eeg
├── m_17_02_pos2a.vhdr
└── m_17_02_pos2a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

In [52]:
### TESTING ###
# test = mne.io.read_raw_brainvision(session_dir+'/'+file_name, preload=True)

In [53]:
# test.plot()

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_17_02/m_17_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_17_02/17_2a_posner_task_2023-06-19_11h49.11.994/17_2a_posner_task_2023-06-19_11h49.11.994.csv
Found '2b' in file: m_17_02/17_2b_posner_task_2023-06-19_12h43.17.233/17_2b_posner_task_2023-06-19_12h43.17.233.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
len(events) 415
len(validity) 100
# of stim events 100
[    0     0 99999]
[76859     0    10]
[78398     0    40]
[79898     0    80]
[83178     0    50]
[83278     0    51]
[84898     0    40]
[86398     0    80]
[89938     0    50]
[90038     0   

/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [334] Keeping  (2023-06-19 12:55:50.608161+00:00 - 2023-06-19 12:55:50.608161+00:00 -> 650.101 - 650.101)
  [335] Keeping  (2023-06-19 12:55:52.768161+00:00 - 2023-06-19 12:55:52.768161+00:00 -> 652.261 - 652.261)
  [336] Keeping  (2023-06-19 12:55:54.268161+00:00 - 2023-06-19 12:55:54.268161+00:00 -> 653.761 - 653.761)
  [337] Keeping  (2023-06-19 12:55:57.298161+00:00 - 2023-06-19 12:55:57.298161+00:00 -> 656.791 - 656.791)
  [338] Keeping  (2023-06-19 12:55:57.398161+00:00 - 2023-06-19 12:55:57.398161+00:00 -> 656.891 - 656.891)
  [339] Keeping  (2023-06-19 12:55:59.269161+00:00 - 2023-06-19 12:55:59.269161+00:00 -> 658.762 - 658.762)
  [340] Keeping  (2023-06-19 12:56:00.769161+00:00 - 2023-06-19 12:56:00.769161+00:00 -> 660.262 - 660.262)
  [341] Keeping  (2023-06-19 12:56:03.328161+00:00 - 2023-06-19 12:56:03.328161+00:00 -> 662.821 - 662.821)
  [342] Keeping  (2023-06-19 12:56:03.429161+00:00 - 2023-06-19 12:56:03.429161+00:00 -> 662.922 - 662.922)
  [343] Keeping  (2023-06-19

In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
415 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 415 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.9s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-06-19 12:46:18.905161+00:00 - 2023-06-19 12:57:58.469161+00:00
  [0] Keeping  (2023-06-19 12:46:18.905161+00:00 - 2023-06-19 12:46:18.905161+00:00 -> 78.398 - 78.398)
  [1] Keeping  (2023-06-19 12:46:20.405161+00:00 - 2023-06-19 12:46:20.405161+00:00 -> 79.898 - 79.898)
  [2] Keeping  (2023-06-19 12:46:23.685161+00:00 - 2023-06-19 12:46:23.685161+00:00 -> 83.178 - 83.178)
  [3] Keeping  (2023-06-19 12:46:23.785161+00:00 - 2023-06-19 12:46:23.785161+00:00 -> 83.278 - 83.278)
  [4] Keeping  (2023-06-19 12:46:25.405161+00:00 - 2023-06-19 12:46:25.405161+00:00 -> 84.898 - 84.898)
  [5] Keeping  (2023-06-19 12:46:26.905161+00:00 - 2023-06-19 12:46:26.905161+00:00 -> 86.398 - 86.398)
  [6] Keeping  (2023-06-19 12:46:30.445161+00:00 - 2023-06-19 12:46:30.445161+00:00 -> 89.938 - 89.938)
  [7] Keeping  (2023-06-19 12:46:30.545161+00:00 - 2023-06-19 12:46

Method,fastica
Fit,83 iterations on raw data (699564 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=699564
    Range : 78398 ... 777961 =     78.398 ...   777.961 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
349 matching events found
No baseline correction applied
0 projection items ac

In [16]:
ica_b.exclude

[4]

In [17]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 1 ICA component
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [18]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_17_02/m_17_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [19]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_17_02
├── m_17_02_pos2a.eeg
├── m_17_02_pos2a.vhdr
├── m_17_02_pos2a.vmrk
├── m_17_02_pos2b.eeg
├── m_17_02_pos2b.vhdr
└── m_17_02_pos2b.vmrk

1 directory, 6 files


# ...

In [20]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b